In [7]:
import sys
!{sys.executable} -m pip install --user sklearn
!{sys.executable} -m pip install --user keras

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 317kB 2.1MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 6.6MB/s ta 0:00:011
    100% |████████████████████████████████| 2.8MB 1.7MB/s ta 0:00:011
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
from sklearn.model_selection import train_test_split
import os

In [4]:
df = pd.read_csv("actions.csv")
df.head(5)

,Unnamed: 0,output,varr
0,0,basketball_shooting,basketball_shooting basketball_shooting basket...
1,1,basketball_shooting,basketball_shooting basketball_shooting biking...
2,2,basketball_shooting,basketball_shooting basketball_shooting basket...
3,3,basketball_shooting,basketball_shooting golf_swing golf_swing bask...
4,4,basketball_shooting,basketball_shooting basketball_shooting basket...


In [25]:
GPU_CUDNN = 0
CPU = 1

In [11]:
#LABELS = np.unique(df.output)
LABELS = [
    "basketball_shooting",
    "biking",
    "diving",
    "golf_swing"
]
n_classes = len(LABELS)

In [14]:
X = df.varr
Y = df.output

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
X_train.head(5)

369    golf_swing basketball_shooting golf_swing golf...
313    golf_swing biking golf_swing golf_swing golf_s...
226    biking diving golf_swing diving diving diving ...
255    basketball_shooting diving diving diving divin...
12     basketball_shooting basketball_shooting basket...
Name: varr, dtype: object

In [24]:
training_data_count = len(X_train)
test_data_count = len(X_test)

n_hidden = 256

training_params = {
    'learning_rate' : 0.0025,
    'lambda_loss_amount': 0.0015,
    'training_iters' : training_data_count * 300,
    'batch_size': 1500,
    'display_iter': 30000
}

In [ ]:
def CNN_model(X, isTraining, num_classes):
    pass

In [27]:
def LSTM_model(X, weights, biases, isTraining, num_classes):
    # Preprocess data input
    
    # Create LSTM cell
    lstm_cell = None;
    if CPU:
        lstm_cell = tf.contrib.rnn.LSTMBlockCell(n_hidden, 
                                                forget_bias=1.0)
    elif GPU_CUDNN:
        lstm_cell = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers=1, 
                                                   num_units=n_hidden,
                                                   kernel_initializer=tf.initializers.random_uniform(-0.01, 0.01),
                                                   bias_initializer=tf.initializers.constant(0))
    else:
        raise Exception('LSTM cell is empty', 'Please fill the CPU/GPU_CUDNN params!!!') 
    
    # Creates a recurrent neural network specified by RNNCell cell.
    lstm_out, _ = tf.contrib.rnn.static_rnn(cell=lstm_cell, 
                                            inputs=X,
                                            dtype=tf.float32)
    # Dropout layer
    dropout = tf.layers.dropout(inputs=lstm_out, 
                               rate=0.5,
                               training=isTraining)
    
    # Fully connected layer
    # weights_initializer is gaussian distribution
    # bias_initializer is constant by zero
    fc = tf.contrib.layers.fully_connected(inputs=dropout,
                                            num_outputs=num_classes,
                                            activation_fn=None,
                                            weights_initializer=tf.initializers.truncated_normal(stddev=0.01),
                                            bias_initializer=tf.initializers.constant(0))
    
    # Batch Norm + Scale layer
    batch_norm = tf.layers.batch_normalization(inputs=fc,
                                                axis=2,
                                                training=isTraining)
    
    # ReLU activation
    relu = tf.nn.relu_layer(batch_norm)
    
    lstm_last_output = outputs[-1]
    return lstm_last_output
    

In [ ]:
def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes 
    # e.g.: 
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS